In [ ]:
### Run this cell before continuing. 
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

In [ ]:
# read data
processed_cleveland <- read_csv("original data/processed.cleveland.data", col_names = F)

In [ ]:
#rename variables
colnames(processed_cleveland) <- c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", 
                                   "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num")

processed_cleveland

In [ ]:
#create the TRAIN SET and TEST SET
set.seed(2000)

cleveland_filter <- processed_cleveland |>
            select(age, chol, thalach, num) |>
            mutate(num = as_factor(num)) |>
            sample_n(300)
cleveland_filter

cleveland_split <- initial_split(cleveland_filter, prop = 0.75, strata = num)
cleveland_train<- training(cleveland_split) 
cleveland_test <- testing(cleveland_split)

# cleveland_vfold <- vfold_cv(cleveland_train, v = 5, strata = num)




In [ ]:
cleveland_recipe <- recipe(num ~ ., data = cleveland_train) |>
                    step_scale(all_predictors()) |>
                    step_center(all_predictors())

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")

# gridvals = tibble(neighbors = seq(from = 1, to = 80, by = 5))

set.seed(2000)

cleve_workflow <- workflow() |>
            add_recipe(cleveland_recipe) |>
            add_model(knn_spec) |>
            fit(data = cleveland_train)

            # tune_grid(resamples = cleveland_vfold, grid = gridvals) 


In [ ]:
# options(repr.plot.height = 5, repr.plot.width = 6)
# cross_val_plot <- cleve_workflow |>
#                 ggplot(aes(x = neighbors, y = mean)) +
#                 geom_point() +
#                 labs(x = "Neighbors", y = "Accuracy")
# cross_val_plot